In [ ]:
#@markdown ####Latest Update: 23-7-2023 04:45 PM UTC - Comfy Colab v1.5 (Support rerunning without reinstall, thanks efreak!)
#@markdown \
#@markdown ## Give your worker a name
#@markdown ##### (If you have used this before, please re-use your worker name)
# The horde url
horde_url = "https://aihorde.net"
worker_name = "Give your worker a cool name" #@param {type:"string"}

if worker_name == "" or worker_name == "Give me a cool worker name":
  print("Please enter a new worker name and resubmit")
#@markdown \
#@markdown ## Enter your API key for the horde
#@markdown ##### (If you havent got one, go to https://aihorde.net/register)
api_key = "" #@param {type:"string"}

if api_key == "":
  print("Please enter an API key and resubmit")
#@markdown \


#@markdown ### Choose a mode
mode = "Simple" #@param ["Simple", "Pick My Model", "Inpainter", "Interrogation"]


#@markdown *Simple Mode = No futher settings required. Press control+f9 to run all the cells to get started. *\
#@markdown *Inpainting Mode = Runs the inpainter model and stable_diffusion* \
#@markdown *Interrogation Mode = Fulfils requests for captions, tags and nsfw* \
#@markdown *Pick A Model Mode = Runs only the model selected below* \
#@markdown ##Run this cell to see a list of models
#@markdown - *'top 1' automatically selects the most popular model (This is the default for simple mode)*
#@markdown - *first three models in the list are models with most images generated in a month*

import requests

models_url = "https://raw.githubusercontent.com/Haidra-Org/AI-Horde-image-model-reference/main/stable_diffusion.json"
if "all_models" not in locals():
  all_models = requests.get(models_url).json()
  model_stats = requests.get("https://aihorde.net/api/v2/stats/img/models").json()
  for name, count in model_stats["month"].items():
    if name in all_models:
      all_models[name]["count"] = count
  inpainting_models = {
      name: params.get("count", 0) for name, params in all_models.items()
      if params["inpainting"]}
  normal_models = {
      name: params.get("count", 0) for name, params in all_models.items()
      if not params["inpainting"]}

if mode == "Inpainter":
  by_popularity = sorted(inpainting_models.items(), key=lambda x: -x[1])
  model_list = [name for name, count in by_popularity]
if mode == "Pick My Model":
  by_popularity = sorted(normal_models.items(), key=lambda x: -x[1])
  popular = [name for name, count in by_popularity[:3]]
  rest = [name for name, count in sorted(by_popularity[3:], key=lambda x: x[0].lower())]
  model_list = ["top 1"] + popular + rest
if 'pick_my_model' in locals():
  model_list = [locals().get('pick_my_model')] + model_list

from markdown import markdown
import ipywidgets as widgets
from ipywidgets import HTML, interactive

def model_chooser(model):
  global pick_my_model
  pick_my_model = model

if mode == "Pick My Model":
  pick_normal_model_widget = interactive(model_chooser, model=model_list)
  display(HTML(markdown("# Pick a model")))
  display(pick_normal_model_widget)
if mode == "Inpainter":
  pick_inpainting_model_widget = interactive(model_chooser, model=model_list)
  display(HTML(markdown("# Pick a model")))
  display(pick_inpainting_model_widget)

In [ ]:
#@markdown ### Define configuration options


#@markdown ### Show Logs
Show_Logs = True #@param {type:"boolean"}
#@markdown *If ticked, this will show a log in the output console for each generation.  If unticked, a flashing cursor will appear until an error occurs and the log is in the AI-Horde-Worker/logs folder*

#@markdown \

#@markdown ## **When finished, press control+f9 to run all the cells, or use the menu at the top (Runtime -> Run all).**
#@markdown ### It will take about 5 minutes for the worker to start processing jobs. Check in to make sure no errors are being encountered every so often.
if mode == "Inpainter":
  allow_painting = True
  dynamic_models = False
  max_power = 22
  models_to_load = [f"{pick_my_model}"]
  models_to_skip = ["stable_diffusion_2.1",  "stable_diffusion_2.0"]
  allow_post_processing = False

elif mode == "Pick My Model":
  allow_painting = False
  dynamic_models = False
  max_power = 22
  models_to_load = [f"{pick_my_model}"]
  models_to_skip = ["stable_diffusion_2.1",  "stable_diffusion_2.0"]
  allow_post_processing = False

else:
  allow_painting = False
  dynamic_models = False
  max_power = 22
  models_to_load = ["top 1"]
  models_to_skip = ["stable_diffusion_inpainting", "stable_diffusion_2.1",  "stable_diffusion_2.0"]
  allow_post_processing = False

###  IGNORE FROM HERE UNLESS YOU KNOW WHAT YOU ARE DOING ###
enable_loras = False
priority_usernames = []
max_threads = 1
nsfw = True
censor_nsfw = False
blacklist = []
censorlist = []
allow_img2img = True
allow_controlnet = False
allow_unsafe_ip = True
number_of_dynamic_models = 0
max_models_to_download = 12
forms = ["caption","nsfw","interrogation","post-process"]

In [ ]:
#@markdown # Download and Install Stable Diffusion for Stable Horde
#@markdown This takes ~5 minutes - please ignore any red errors telling you to restart the runtime after installing a new version of something

%cd /content/

if 'install_completed' not in locals():
    !git clone https://github.com/db0/AI-Horde-Worker.git

    %cd /content/AI-Horde-Worker
    !pip install -r requirements.txt

install_completed=True



In [ ]:
#@markdown # Start Stable Horde worker
#@markdown For maximum runtime, leave this browser tab running and clear the logs every 15-30 mins by hitting the X in the top left corner of the logs (will appear when hovering over the three dots)
import os

%cd /content/AI-Horde-Worker

!rm -rf bridgeData.yaml

from yaml import load, dump
def make_yaml_sublist(list_to_convert: list[str]):
  sublist_yaml = dump(list_to_convert)
  sublist_yaml = "\n" + sublist_yaml
  return sublist_yaml

if "api_key" not in locals():
  print()
  print("*" * 80)
  print("ERROR: You haven't set an API key... double check the first cell at the top of the page.")
  print("*" * 80)
  print()

data = f"""horde_url: "{horde_url}"
api_key: "{api_key}"
priority_usernames: []
max_threads: {max_threads}
queue_size: 1
require_upfront_kudos: false
dreamer_name: "{worker_name}"
max_power: {max_power}
nsfw: {nsfw.__str__().lower()}
censor_nsfw: false
blacklist: {blacklist}
censorlist: {censorlist}
allow_img2img: {allow_img2img.__str__().lower()}
allow_painting: {allow_painting.__str__().lower()}
allow_unsafe_ip: true
allow_post_processing: {allow_post_processing.__str__().lower()}
allow_controlnet: false
dynamic_models: false
number_of_dynamic_models: 0
max_models_to_download: 10
stats_output_frequency: 30
cache_home: "./"
always_download: true
temp_dir: "./tmp"
disable_terminal_ui: true
vram_to_leave_free: "40%"
ram_to_leave_free: "80%"
disable_disk_cache: false
models_to_load: {make_yaml_sublist(models_to_load)}
models_to_skip: {make_yaml_sublist(models_to_skip)}
forms:
  - "caption"
  - "nsfw"
  - "interrogation"
  - "post-process"
allow_lora: {enable_loras.__str__().lower()}
"""

with open("bridgeData.yaml", "w") as text_file:
  text_file.write(data)

!export LOW_VRAM_MODE=0
if mode == "Interrogation":
  if Show_Logs:
    !python bridge_alchemy.py -y
  else:
    print ("Your worker has started and is running while the cursor below is flashing")
    !python bridge_alchemy.py -y -q

else:
  if Show_Logs:
    !python bridge_stable_diffusion.py -y
  else:
    print ("Your worker has started and is running while the cursor below is flashing")
    !python bridge_stable_diffusion.py -y -q